# Fincomun Enrichment

> Carlos Alvarez Aparicio

> 15/06/2022

# IMPORT PACKAGES

In [41]:
import pandas as pd
import gcsfs
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from hocelot.dlutils import get_col_feats,open
from datetime import datetime
sns.set_style("whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 400)
pd.set_option('max_colwidth',400)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from hocelot.dlmetadata.hocelot_types import HOCELOT_DTYPES, HOCELOT_FTYPES
from hocelot.dlmetadata.log_transformable_features import LOG_TRANSFORMABLE_1, LOG_TRANSFORMABLE_2
from hocelot.rdt.models import RDTransformer
import joblib
import json

In [42]:
import os
import pickle
import joblib
import re
import string
import subprocess
import random
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import lightgbm
from xgboost import XGBClassifier
from multiprocessing import cpu_count
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score, roc_curve

from hocelot.dlutils import get_col_feats, read_raw_data
from hocelot.dlutils import open as open_dlutils # open for saving PKL
#from hocelot.dlutils.miscellaneous import RANDOM_SEED, hocelot_colors
from hocelot.rdt import RDTransformer
from hocelot.dlmodels import XGBClassifierSelector
from hocelot.dlplots import ModelResultsGrapher, ModelExplainabilityGrapher, DefaultGrapher
#from hocelot.dlplots.classification.general_plots import plot_target_stability
from hocelot.dlmetadata import HOCELOT_DTYPES, HOCELOT_FTYPES, LOG_TRANSFORMABLE_1
from hocelot.dlutils.raw_data_reader import read_raw_data

from hocelot.dlutils import open

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_rows', 100)

HOCELOT_YELLOW = '#FDD226'
HOCELOT_BLACK = '#000000'
HOCELOT_RED = '#FF5753'
HOCELOT_BLUE = '#2B3252'
HOCELOT_WHITE = '#FFFFFF'

%matplotlib inline
warnings.filterwarnings('ignore')

# LOAD DATASET

In [43]:
data_clientes_con_target = pd.read_csv('gs://hocelot_mx_analytics/mexico/progress/mx-00007-Came/data_Came_pre_enrich_con_target_v0.csv',sep=';',index_col=False)
print(data_clientes_con_target.shape)
data_clientes_con_target.head()

(13262, 17)


,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,pk,Tasa_medio,Plazo_medio,Monto_medio,EstadoCivil,Default_90_dias_pers
0,48993,VIANNEY CHABLE MORA,2002-08-12,CAMV020812MMCHRNA2,MANUEL ALTAMIRANO,2,ARTESANOS,ESTADO DE MEXICO,ESTADO DE MEXICO,vianneychable001@gmail.com,5588825117,0,106.8000,12.0000,21600.0000,UNIÓN LIBRE,1
1,141508,FRANCISCO LOPEZ ROQUE,1957-10-21,LORF571021HCLPQR04,HIDALGO,1701,VICTORIA,NUEVO LAREDO,TAMAULIPAS,franciscolopezroque@gmail.com,8677414704,1,120.0000,12.0000,21440.0000,SOLTERO,0
2,172718,MA DEL PILAR FLORES SERAFIN,1953-10-12,FOSP531012MMCLRL00,CLL LEONA VICARIO,34,SAN VICENTE CHICOLOAPAN DE JUÁREZ CENTRO,CHICOLOAPAN,ESTADO DE MÉXICO,jorgemilan1971@gmail.com,5585041612,2,106.8000,7.0000,22050.0000,VIUDO,0
3,274775,EDWIN FERNANDO DEL CANTO FLORES,1991-08-21,CAFE910821HMCNLD08,AV PROMESAS,36,TEPETITLAN,ESTADO DE MEXICO,ESTADO DE MEXICO,edwin_delcanto@hotmail.com,5525527792,3,90.0000,15.0000,51380.0000,SOLTERO,0
4,365550,JUDIHT CORTES RAMIREZ,1983-10-26,CORJ831026MOCRMD09,C JUSTO SIERRA,402,TEPEYAC,VILLA DE TUTUTEPEC,OAXACA,cortesjudiht18@gmail.com,9541658986,4,106.8000,12.0000,26440.0000,CASADO,0


In [44]:
data_clientes_con_target['pk'] = data_clientes_con_target['pk'].astype(str)

## CARGAMOS EL FICHERO YA ENRIQUECIDO

In [45]:
with open('gs://hocelot_mx_analytics/mexico/progress/mx-00007-Came/response_Came_pre_enrich_v0.json') as file:
    data = json.load(file)
df_enriched=pd.json_normalize(data)

In [46]:
df_enriched=df_enriched.dropna(axis=1, how='all')
print(df_enriched.shape)
df_enriched.head()

(13262, 139)


,pk,personalData.identityDocument,personalData.fullname,personalData.gender,personalData.birthdate,personalData.age,personalData.email,personalData.phone,emailValidation.valid,emailValidation.email,emailValidation.domain,emailValidation.syntaxError,emailValidation.domainError,emailValidation.provider,emailValidation.disposable,emailValidation.freemail,emailVerification.verified,emailVerification.status,emailVerification.deferred,emailVerification.catchAll,phoneValidation.valid,phoneValidation.type,phoneValidation.internationalNumber,phoneValidation.localNumber,phoneValidation.country,phoneValidation.countryCode,phoneValidation.mobile,phoneVerification.verified,phoneVerification.status,phoneVerification.ported,phoneVerification.roaming,phoneVerification.imsi,phoneVerification.mcc,phoneVerification.mnc,phoneVerification.msin,phoneVerification.msc,phoneVerification.currentNetwork,phoneVerification.originNetwork,phoneVerification.portedNetwork,geoCheck.score,geoCheck.parsedAddress.level1,geoCheck.parsedAddress.level2,geoCheck.parsedAddress.sublevel3,geoCheck.parsedAddress.zipCode,geoCheck.parsedAddress.roadName,geoCheck.parsedAddress.roadNumber,geoCheck.parsedAddress.latitude,geoCheck.parsedAddress.longitude,geoCheck.parsedAddress.additionalData,geoCheck.cvegeos,economics.id,economics.rent,economics.typeTable,economics.personalData.dependency,economics.personalData.work,economics.personalData.possession,economics.personalData.levelStudy,economics.personalData.maritalStatus,economics.personalData.ageInterval,economics.personalData.ageIntervalMin,economics.personalData.ageIntervalMax,economics.personalData.probCouple,economics.personalData.probDegree,economics.personalData.probDivorced,economics.personalData.probDoctorate,economics.personalData.gender,economics.personalData.bossEducation,economics.personalData.probHighschool,economics.personalData.probMarried,economics.personalData.probMasters,economics.personalData.probNormal,economics.personalData.probNpLevels,economics.personalData.probNpMaritals,economics.personalData.probPreschool,economics.personalData.probPrimary,economics.personalData.probProfessional,economics.personalData.probSecondary,economics.personalData.probSeparated,economics.personalData.probSingle,economics.personalData.probWidower,economics.personalData.probWithoutStudies,economics.familyData.numberOfBabies,economics.familyData.numberOfChild,economics.familyData.numberOfDep,economics.familyData.numberOfDepYoung,economics.familyData.numberOfIndep,economics.familyData.numberOfIndepYoung,economics.familyData.numberOfParents,economics.familyData.numberOfYoungs,economics.familyData.typeMember,economics.housingData.car,economics.housingData.numberHouseholdCars,economics.housingData.numberHouseholdMembers,economics.housingData.numberHouseholdMembersNoOutliers,economics.housingData.numberHouseholdPickups,economics.housingData.numberHouseholdVans,economics.housingData.numberIncandescentLights,economics.housingData.numberLowConsumptionLights,economics.housingData.numberRooms,economics.housingData.bathShower,economics.housingData.bathShowerToilet,economics.housingData.housingMaterial,economics.housingData.housingType,economics.housingData.internetConnection,economics.housingData.locationSize,economics.housingData.codEntity,economics.housingData.codMunicipality,economics.housingData.codLocality,economics.housingData.latitude,economics.housingData.longitude,economics.householdEconomy.probSaving,economics.householdEconomy.probExpenseAlcoholAndTobacco,economics.householdEconomy.probExpenseClothesAndFootwear,economics.householdEconomy.probExpenseEducation,economics.householdEconomy.probExpenseFinancial,economics.householdEconomy.probExpenseFood,economics.householdEconomy.probExpenseHealthCare,economics.householdEconomy.probExpenseHealthInsurance,economics.householdEconomy.probExpenseHomeRepair,economics.householdEconomy.probExpenseHousesAndSupplies,economics.householdEconomy.probExpenseLeisure,economics.householdEconomy.probExpen

In [47]:
df_enriched['pk'][0]

'0'

In [48]:
data_clientes_con_target.merge(df_enriched, how='inner', on='pk').shape

(13262, 155)

In [49]:
data_clientes_con_target['pk'].value_counts()

0        1
8845     1
8835     1
8836     1
8837     1
        ..
4423     1
4424     1
4425     1
4426     1
13261    1
Name: pk, Length: 13262, dtype: int64

In [50]:
df_enriquecido_total['pk'].value_counts()

2662     3
7416     3
7608     2
6983     2
7002     2
        ..
4474     1
4472     1
4476     1
4475     1
13261    1
Name: pk, Length: 13262, dtype: int64

In [51]:
df_enriquecido_total = data_clientes_con_target.merge(df_enriched, how='right', on='pk')
print(df_enriquecido_total.shape)
df_enriquecido_total.head()

(13262, 155)


,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,pk,Tasa_medio,Plazo_medio,Monto_medio,EstadoCivil,Default_90_dias_pers,personalData.identityDocument,personalData.fullname,personalData.gender,personalData.birthdate,personalData.age,personalData.email,personalData.phone,emailValidation.valid,emailValidation.email,emailValidation.domain,emailValidation.syntaxError,emailValidation.domainError,emailValidation.provider,emailValidation.disposable,emailValidation.freemail,emailVerification.verified,emailVerification.status,emailVerification.deferred,emailVerification.catchAll,phoneValidation.valid,phoneValidation.type,phoneValidation.internationalNumber,phoneValidation.localNumber,phoneValidation.country,phoneValidation.countryCode,phoneValidation.mobile,phoneVerification.verified,phoneVerification.status,phoneVerification.ported,phoneVerification.roaming,phoneVerification.imsi,phoneVerification.mcc,phoneVerification.mnc,phoneVerification.msin,phoneVerification.msc,phoneVerification.currentNetwork,phoneVerification.originNetwork,phoneVerification.portedNetwork,geoCheck.score,geoCheck.parsedAddress.level1,geoCheck.parsedAddress.level2,geoCheck.parsedAddress.sublevel3,geoCheck.parsedAddress.zipCode,geoCheck.parsedAddress.roadName,geoCheck.parsedAddress.roadNumber,geoCheck.parsedAddress.latitude,geoCheck.parsedAddress.longitude,geoCheck.parsedAddress.additionalData,geoCheck.cvegeos,economics.id,economics.rent,economics.typeTable,economics.personalData.dependency,economics.personalData.work,economics.personalData.possession,economics.personalData.levelStudy,economics.personalData.maritalStatus,economics.personalData.ageInterval,economics.personalData.ageIntervalMin,economics.personalData.ageIntervalMax,economics.personalData.probCouple,economics.personalData.probDegree,economics.personalData.probDivorced,economics.personalData.probDoctorate,economics.personalData.gender,economics.personalData.bossEducation,economics.personalData.probHighschool,economics.personalData.probMarried,economics.personalData.probMasters,economics.personalData.probNormal,economics.personalData.probNpLevels,economics.personalData.probNpMaritals,economics.personalData.probPreschool,economics.personalData.probPrimary,economics.personalData.probProfessional,economics.personalData.probSecondary,economics.personalData.probSeparated,economics.personalData.probSingle,economics.personalData.probWidower,economics.personalData.probWithoutStudies,economics.familyData.numberOfBabies,economics.familyData.numberOfChild,economics.familyData.numberOfDep,economics.familyData.numberOfDepYoung,economics.familyData.numberOfIndep,economics.familyData.numberOfIndepYoung,economics.familyData.numberOfParents,economics.familyData.numberOfYoungs,economics.familyData.typeMember,economics.housingData.car,economics.housingData.numberHouseholdCars,economics.housingData.numberHouseholdMembers,economics.housingData.numberHouseholdMembersNoOutliers,economics.housingData.numberHouseholdPickups,economics.housingData.numberHouseholdVans,economics.housingData.numberIncandescentLights,economics.housingData.numberLowConsumptionLights,economics.housingData.numberRooms,economics.housingData.bathShower,economics.housingData.bathShowerToilet,economics.housingData.housingMaterial,economics.housingData.housingType,economics.housingData.internetConnection,economics.housingData.locationSize,economics.housingData.codEntity,economics.housingData.codMunicipality,economics.housingData.codLocality,economics.housingData.latitude,economics.housingData.longitude,economics.householdEconomy.probSaving,economics.householdEconomy.probExpenseAlcoholAndTobacco,economics.householdEconomy.probExpenseClothesAndFootwear,economics.householdEconomy.probExpenseEducation,economics.householdEconomy.probExpenseFinancial,economics.householdEconomy.probExpenseFood,economics.householdEconomy.probExpenseHealthCare,economics.householdEconomy.probExpenseHealthInsurance,economics.householdEconomy.prob

In [52]:
df_enriquecido_total.drop(columns=['geoCheck.parsedAddress.additionalData','geoCheck.cvegeos'], inplace=True)

In [53]:
pd.set_option('display.max_rows', None)
get_col_feats(df_enriquecido_total)

100%|██████████| 153/153 [00:00<00:00, 469.82it/s]


,dtype,sample,n_missing,perc_missing,n_unique
NoCliente,int64,48993,0,0.000000,13260
Nombre,object,VIANNEY CHABLE MORA,0,0.000000,13251
FechaNacimiento,object,2002-08-12,0,0.000000,9109
CURP,object,CAMV020812MMCHRNA2,0,0.000000,13260
Calle,object,MANUEL ALTAMIRANO,0,0.000000,9765
NumExt,object,2,2,0.015081,3197
Colonia,object,ARTESANOS,0,0.000000,5130
Ciudad,object,ESTADO DE MEXICO,0,0.000000,575
Estado,object,ESTADO DE MEXICO,0,0.000000,38
Email,object,vianneychable001@gmail.com,0,0.000000,13259


In [54]:
for col in df_enriquecido_total.columns:
    if df_enriquecido_total[col].nunique() == 1:
        print(col)
        df_enriquecido_total = df_enriquecido_total.drop(columns=col)

emailVerification.deferred
phoneValidation.type
phoneValidation.country
phoneValidation.countryCode
phoneValidation.mobile
phoneVerification.roaming
phoneVerification.msin


In [55]:
print(df_enriquecido_total.shape)
df_enriquecido_total.head(1)

(13262, 146)


,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,pk,Tasa_medio,Plazo_medio,Monto_medio,EstadoCivil,Default_90_dias_pers,personalData.identityDocument,personalData.fullname,personalData.gender,personalData.birthdate,personalData.age,personalData.email,personalData.phone,emailValidation.valid,emailValidation.email,emailValidation.domain,emailValidation.syntaxError,emailValidation.domainError,emailValidation.provider,emailValidation.disposable,emailValidation.freemail,emailVerification.verified,emailVerification.status,emailVerification.catchAll,phoneValidation.valid,phoneValidation.internationalNumber,phoneValidation.localNumber,phoneVerification.verified,phoneVerification.status,phoneVerification.ported,phoneVerification.imsi,phoneVerification.mcc,phoneVerification.mnc,phoneVerification.msc,phoneVerification.currentNetwork,phoneVerification.originNetwork,phoneVerification.portedNetwork,geoCheck.score,geoCheck.parsedAddress.level1,geoCheck.parsedAddress.level2,geoCheck.parsedAddress.sublevel3,geoCheck.parsedAddress.zipCode,geoCheck.parsedAddress.roadName,geoCheck.parsedAddress.roadNumber,geoCheck.parsedAddress.latitude,geoCheck.parsedAddress.longitude,economics.id,economics.rent,economics.typeTable,economics.personalData.dependency,economics.personalData.work,economics.personalData.possession,economics.personalData.levelStudy,economics.personalData.maritalStatus,economics.personalData.ageInterval,economics.personalData.ageIntervalMin,economics.personalData.ageIntervalMax,economics.personalData.probCouple,economics.personalData.probDegree,economics.personalData.probDivorced,economics.personalData.probDoctorate,economics.personalData.gender,economics.personalData.bossEducation,economics.personalData.probHighschool,economics.personalData.probMarried,economics.personalData.probMasters,economics.personalData.probNormal,economics.personalData.probNpLevels,economics.personalData.probNpMaritals,economics.personalData.probPreschool,economics.personalData.probPrimary,economics.personalData.probProfessional,economics.personalData.probSecondary,economics.personalData.probSeparated,economics.personalData.probSingle,economics.personalData.probWidower,economics.personalData.probWithoutStudies,economics.familyData.numberOfBabies,economics.familyData.numberOfChild,economics.familyData.numberOfDep,economics.familyData.numberOfDepYoung,economics.familyData.numberOfIndep,economics.familyData.numberOfIndepYoung,economics.familyData.numberOfParents,economics.familyData.numberOfYoungs,economics.familyData.typeMember,economics.housingData.car,economics.housingData.numberHouseholdCars,economics.housingData.numberHouseholdMembers,economics.housingData.numberHouseholdMembersNoOutliers,economics.housingData.numberHouseholdPickups,economics.housingData.numberHouseholdVans,economics.housingData.numberIncandescentLights,economics.housingData.numberLowConsumptionLights,economics.housingData.numberRooms,economics.housingData.bathShower,economics.housingData.bathShowerToilet,economics.housingData.housingMaterial,economics.housingData.housingType,economics.housingData.internetConnection,economics.housingData.locationSize,economics.housingData.codEntity,economics.housingData.codMunicipality,economics.housingData.codLocality,economics.housingData.latitude,economics.housingData.longitude,economics.householdEconomy.probSaving,economics.householdEconomy.probExpenseAlcoholAndTobacco,economics.householdEconomy.probExpenseClothesAndFootwear,economics.householdEconomy.probExpenseEducation,economics.householdEconomy.probExpenseFinancial,economics.householdEconomy.probExpenseFood,economics.householdEconomy.probExpenseHealthCare,economics.householdEconomy.probExpenseHealthInsurance,economics.householdEconomy.probExpenseHomeRepair,economics.householdEconomy.probExpenseHousesAndSupplies,economics.householdEconomy.probExpenseLeisure,economics.householdEconomy.probExpenseOthers,economics.householdEconomy.probExpensePurchaseExp,economics.ho

In [56]:
df_enriquecido_total.to_csv('gs://hocelot_mx_analytics/mexico/progress/mx-00007-Came/data_Came_post_enrich_con_target_v0.csv',sep=';', index=False)

In [57]:
print(df_enriquecido_total.shape)
df_enriquecido_total.head()

(13262, 146)


,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,pk,Tasa_medio,Plazo_medio,Monto_medio,EstadoCivil,Default_90_dias_pers,personalData.identityDocument,personalData.fullname,personalData.gender,personalData.birthdate,personalData.age,personalData.email,personalData.phone,emailValidation.valid,emailValidation.email,emailValidation.domain,emailValidation.syntaxError,emailValidation.domainError,emailValidation.provider,emailValidation.disposable,emailValidation.freemail,emailVerification.verified,emailVerification.status,emailVerification.catchAll,phoneValidation.valid,phoneValidation.internationalNumber,phoneValidation.localNumber,phoneVerification.verified,phoneVerification.status,phoneVerification.ported,phoneVerification.imsi,phoneVerification.mcc,phoneVerification.mnc,phoneVerification.msc,phoneVerification.currentNetwork,phoneVerification.originNetwork,phoneVerification.portedNetwork,geoCheck.score,geoCheck.parsedAddress.level1,geoCheck.parsedAddress.level2,geoCheck.parsedAddress.sublevel3,geoCheck.parsedAddress.zipCode,geoCheck.parsedAddress.roadName,geoCheck.parsedAddress.roadNumber,geoCheck.parsedAddress.latitude,geoCheck.parsedAddress.longitude,economics.id,economics.rent,economics.typeTable,economics.personalData.dependency,economics.personalData.work,economics.personalData.possession,economics.personalData.levelStudy,economics.personalData.maritalStatus,economics.personalData.ageInterval,economics.personalData.ageIntervalMin,economics.personalData.ageIntervalMax,economics.personalData.probCouple,economics.personalData.probDegree,economics.personalData.probDivorced,economics.personalData.probDoctorate,economics.personalData.gender,economics.personalData.bossEducation,economics.personalData.probHighschool,economics.personalData.probMarried,economics.personalData.probMasters,economics.personalData.probNormal,economics.personalData.probNpLevels,economics.personalData.probNpMaritals,economics.personalData.probPreschool,economics.personalData.probPrimary,economics.personalData.probProfessional,economics.personalData.probSecondary,economics.personalData.probSeparated,economics.personalData.probSingle,economics.personalData.probWidower,economics.personalData.probWithoutStudies,economics.familyData.numberOfBabies,economics.familyData.numberOfChild,economics.familyData.numberOfDep,economics.familyData.numberOfDepYoung,economics.familyData.numberOfIndep,economics.familyData.numberOfIndepYoung,economics.familyData.numberOfParents,economics.familyData.numberOfYoungs,economics.familyData.typeMember,economics.housingData.car,economics.housingData.numberHouseholdCars,economics.housingData.numberHouseholdMembers,economics.housingData.numberHouseholdMembersNoOutliers,economics.housingData.numberHouseholdPickups,economics.housingData.numberHouseholdVans,economics.housingData.numberIncandescentLights,economics.housingData.numberLowConsumptionLights,economics.housingData.numberRooms,economics.housingData.bathShower,economics.housingData.bathShowerToilet,economics.housingData.housingMaterial,economics.housingData.housingType,economics.housingData.internetConnection,economics.housingData.locationSize,economics.housingData.codEntity,economics.housingData.codMunicipality,economics.housingData.codLocality,economics.housingData.latitude,economics.housingData.longitude,economics.householdEconomy.probSaving,economics.householdEconomy.probExpenseAlcoholAndTobacco,economics.householdEconomy.probExpenseClothesAndFootwear,economics.householdEconomy.probExpenseEducation,economics.householdEconomy.probExpenseFinancial,economics.householdEconomy.probExpenseFood,economics.householdEconomy.probExpenseHealthCare,economics.householdEconomy.probExpenseHealthInsurance,economics.householdEconomy.probExpenseHomeRepair,economics.householdEconomy.probExpenseHousesAndSupplies,economics.householdEconomy.probExpenseLeisure,economics.householdEconomy.probExpenseOthers,economics.householdEconomy.probExpensePurchaseExp,economics.ho